In [423]:
# from PyQt5 import QtCore, QtGui, QtWidgets
# from testcombobox import Ui_mainWindow
from datetime import datetime
import sys
import urllib.request
import pandas as pd
import numpy as np
# from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
#Инициализация нашего окна
# app = QtWidgets.QApplication(sys.argv)
#Форма
# mainWindow = QtWidgets.QMainWindow()
# ui = Ui_mainWindow()
# ui.setupUi(mainWindow)
# mainWindow.show()
#Логика программы
def Download():
     
#     vstat = ui.lineediturl.text()
#     page = urllib.request.urlopen(vstat)
    
    vstat = "http://mgr.imces.ru/stdpub/kavkaz/vladikavkaz/26727EE0BF9F15BA_060617_074930.mgr"
    page = urllib.request.urlopen(vstat)
    
#     vstat = '1.gr1'
#     page = open(vstat, "rb")
    
    
    content = page.read()
    type_file = vstat.split('.')
#-----------------------------------------------------------------------------------------------------------------    
    if type_file[-1] == 'gr1':
        width = 8 # shirina fayla gr1
        numpy_data_polojitelnie = np.frombuffer(content, dtype=np.dtype('B')).reshape(int(len(content) / width), width)
        length_column = int(len(content) / width) - 2 # вычитаем -2 так как первые 16 байтов являтся заголовком, каждая строка по 8 байтам и не читаем первые две строки
        numpy_date = np.array(numpy_data_polojitelnie[2:, :4], dtype=np.uint32)
    
    if type_file[-1] == 'gr2':
        width = 12 # shirina fayla gr2
        numpy_data_polojitelnie = np.frombuffer(content, dtype=np.dtype('B')).reshape(int(len(content) / width), width)
        length_column = int(len(content) / width) - 1 # вычитаем -1 так как первые 12 байтов являтся заголовком 
        numpy_date = np.array(numpy_data_polojitelnie[1:, :4], dtype=np.uint32)
    
    if type_file[-1] == 'mgr':
        width = 13 # shirina fayla mgr
        numpy_data_polojitelnie = np.frombuffer(content, dtype=np.dtype('B')).reshape(int(len(content) / width), width)
        length_column = int(len(content) / width) - 1 # вычитаем -1 так как первые 13 байтов являтся заголовком
        numpy_date_temp = np.array(numpy_data_polojitelnie[1:, :3], dtype=np.uint32)
        numpy_date = []
        b = np.array(numpy_date_temp, dtype='U2')
        for i in range(len(numpy_date_temp)):
            numpy_date.append(b[i,0]+':' + b[i,1] + ':' + b[i,2])
        array_date_obichniy_format = np.array(numpy_date,dtype='U7')
#-----------------------------------------------------------------------------------------------------------------  
    if width == 8: # это означает что ширина = 8 то есть фалй gr1
        numpy_H1_amp_count = numpy_data_polojitelnie[2:, 4:6]
        numpy_H1_amplituda = numpy_data_polojitelnie[2:, 6:8]
    
    if width == 12: # это означает что ширина = 12 то есть фалй gr2 
        numpy_H1_amp_count = numpy_data_polojitelnie[1:, 4:6]
        numpy_H1_amplituda = numpy_data_polojitelnie[1:, 6:8]
        numpy_H2_amp_count = numpy_data_polojitelnie[1:, 8:10]
        numpy_H2_amplituda = numpy_data_polojitelnie[1:, 10:12]
    
    if width == 13: # это означает что ширина = 13 то есть фалй mgr
        numpy_H1_amp_count = numpy_data_polojitelnie[1:, 3:5]
        numpy_H2_amp_count = numpy_data_polojitelnie[1:, 5:7]
        numpy_E_amp_count  = numpy_data_polojitelnie[1:, 7:9]
        numpy_H1_amplituda = numpy_data_polojitelnie[1:, 9:11]
        numpy_H2_amplituda = numpy_data_polojitelnie[1:, 11:13]
        
#-----------------------------------------------------------------------------------------------------------------        
    def date_v_sekundax(massiv):
        s = np.zeros(length_column, dtype='uint64')
        for i in range(length_column):
            s[i] = massiv[i, 3] * (pow(16, 6)) + massiv[i, 1] * (pow(16, 4)) + massiv[i, 2] * (pow(16, 2)) + massiv[i, 0]
        return s
    
    if width == 8 or width == 12:
        array_date_v_sekundax = date_v_sekundax(numpy_date)

    def data_obichniy_format(massiv):
        s = np.zeros(length_column, dtype='<U40')  # "<U36"dlina simvolov
        for i in range(len(massiv)):
            s[i] = datetime.fromtimestamp(massiv[i] - 2209032000).strftime("%A, %B %d, %Y %I:%M:%S")
        return s

    if width == 8 or width == 12:
            array_date_obichniy_format = data_obichniy_format(array_date_v_sekundax)
    

    def func_join_2_column(massiv):
        s = np.zeros(length_column, dtype='uint32')
        for i in range(length_column):
            s[i] = massiv[i, 0] + massiv[i, 1] * 16
        return s
#-----------------------------------------------------------------------------------------------------------------
    if width == 8:
        array_H1_amp_count = func_join_2_column(numpy_H1_amp_count)
        array_H1_amplituda = func_join_2_column(numpy_H1_amplituda)
        
        dataset = pd.DataFrame(
        {'Time': array_date_obichniy_format, 'ImpulseNS': array_H1_amp_count, 'AmplitudaNS': array_H1_amplituda})
    
    if width == 12:
        array_H1_amp_count = func_join_2_column(numpy_H1_amp_count)
        array_H1_amplituda = func_join_2_column(numpy_H1_amplituda)
        array_H2_amp_count = func_join_2_column(numpy_H2_amp_count)
        array_H2_amplituda = func_join_2_column(numpy_H2_amplituda)
        
        dataset = pd.DataFrame(
        {'Time': array_date_obichniy_format, 'ImpulseNS': array_H1_amp_count, 'AmplitudaNS': array_H1_amplituda, 'ImpulseWE': array_H2_amp_count,
         'AmplitudaWE': array_H2_amplituda})
        
    if width == 13:
        array_H1_amp_count = func_join_2_column(numpy_H1_amp_count)
        array_H2_amp_count = func_join_2_column(numpy_H2_amp_count)
        array_E_amp_count  = func_join_2_column(numpy_E_amp_count )
        array_H1_amplituda = func_join_2_column(numpy_H1_amplituda)
        array_H2_amplituda = func_join_2_column(numpy_H2_amplituda)
        
        dataset = pd.DataFrame({'Time': array_date_obichniy_format, 'ImpulseNS': array_H1_amp_count, 'AmplitudaNS': array_H1_amplituda, 'ImpulseWE': array_H2_amp_count,
         'AmplitudaWE': array_H2_amplituda})
#-----------------------------------------------------------------------------------------------------------------   
    
    return dataset

# ui.downloadpush.clicked.connect( Download )

def peredacha():
    x = ui.comboBox.currentText()
    if x == "Импульс (Север-Юг)":
        p = Download()['ImpulseNS']
    if x == "Амплитуда (Север-Юг)":
        p = Download()['AmplitudaNS']
    if x == "Импульс (Запад-Восток)":
        p = Download()['ImpulseWE']
    if x == "Амплитуда (Запад-Восток)":
        p = Download()['AmplitudaWE']
    return p

# def colichestvo():
#     r1 = peredacha().count()
#     ui.lineEditstat.setText(str(r1))
# ui.countpush.clicked.connect( colichestvo )

# def sredneeznachenie(self):
#     r2 = peredacha().mean()
#     ui.lineEditstat.setText(str(r2))

# ui.meanpush.clicked.connect( sredneeznachenie )

# def mediana(self):
#     r3 = peredacha().median()
#     ui.lineEditstat.setText(str(r3))

# ui.medianpush.clicked.connect( mediana )

# def maingraph():
#     sst = peredacha()
#     n = len(sst)
#     time = np.arange(n)
#     ui.MplWidget.canvas.axes.clear()
#     ui.MplWidget.canvas.axes.plot(time, sst)
#     ui.MplWidget.canvas.axes.set_title('Исходный график', fontsize=24)
#     ui.MplWidget.canvas.draw()

# ui.maingraphpush.clicked.connect( maingraph )

# def wavelet():
#     from scipy import signal
#     import matplotlib.pyplot as plt
#     widths = np.arange(1, 61)
#     cwtmatr = signal.cwt(peredacha(), signal.ricker, widths)
#     fig = plt.figure(figsize=(20, 10))
#     ui.MplWidget.canvas.axes.set_title('Вейвлет преобразование Мексиканская шляпа')
#     plt.imshow(cwtmatr, extent=[-1, 365, -1, 1], cmap='PRGn', aspect='auto', vmax=abs(cwtmatr).max(),
#                vmin=-abs(cwtmatr).max())
#     plt.xlabel('Дни', fontsize=20)
#     plt.ylabel('Амплитуда', fontsize=20)
#     ui.MplWidget.canvas.draw()

# ui.wavelet1.clicked.connect( wavelet )


# #Окно
# sys.exit(app.exec_())


In [425]:
Download()['ImpulseNS']

0     76
1     97
2     61
3     60
4     66
5     71
6     36
7     62
8     34
9     56
10    34
11    53
12    77
13    11
14    37
15    55
16    50
17    61
18    51
19    46
20    26
Name: ImpulseNS, dtype: uint32